In [ ]:
!uv pip install pandas, geopandas

In [ ]:
!uv pip install duckdb -U # upgrade to latest version

# DuckDB spatial extension and st_readOSM
## Benchmark: extracting all roads under construction for all of Germany 
### Hardware: M3 Max with 128GB RAM, 20 cores, and 40 threads.

# 0. Download Germany pbf from GeoFabrik from https://download.geofabrik.de/europe.html
`wget https://download.geofabrik.de/europe/germany-latest.osm.pbf`

# 1. Query PBF File with DuckDB -> Pandas df

In [ ]:
import duckdb
import geopandas as gpd
import pandas as pd
import os

# --- 1. Setup ---
pbf_file = "germany-latest.osm.pbf"

if not os.path.exists(pbf_file):
    raise FileNotFoundError(f"The file '{pbf_file}' was not found.")

# --- 2. Optimized High-Performance DuckDB Query ---
query = f"""
PRAGMA enable_optimizer;
WITH
    -- Step 1: Filter ways based on tags first. This is the most selective filter.
    -- The optimizer will push this down to the OSM reader.
    filtered_ways AS (
        SELECT id, tags, refs
        FROM st_readOSM('{pbf_file}')
        WHERE
            kind = 'way'
            AND tags['highway'] IS NOT NULL
            AND tags['construction'] IS NOT NULL
            -- It's good practice to also exclude proposed/planned features if not wanted
            -- AND tags['construction'] NOT IN ('proposed', 'planned')
            AND array_length(refs) >= 2
    ),

    -- Step 2: Get only the nodes that are referenced by our filtered ways.
    -- This is more direct than creating a separate CTE for IDs.
    nodes AS (
        SELECT id, ST_Point(lon, lat) AS geom
        FROM st_readOSM('{pbf_file}')
        WHERE
            kind = 'node'
            AND id IN (SELECT UNNEST(refs) FROM filtered_ways)
    ),
    
    -- Step 3: Unnest way-node relationships for ordering
    way_nodes AS (
        SELECT
            w.id AS way_id,
            w.tags,
            UNNEST(w.refs) AS node_id,
            UNNEST(generate_series(1, array_length(w.refs))) AS node_order
        FROM filtered_ways w
    )

-- Step 4: Final geometry construction
SELECT
    wn.way_id AS id,
    wn.tags,
    ST_AsWKB(ST_MakeLine(LIST(n.geom ORDER BY wn.node_order))) AS geom_wkb
FROM way_nodes wn
INNER JOIN nodes n ON wn.node_id = n.id
GROUP BY wn.way_id, wn.tags;
"""

# --- 3. Execute with Optimized Configuration ---
print("Connecting to DuckDB with optimized settings...")

with duckdb.connect(database=':memory:') as con:
    # Optimize DuckDB for M3 Max performance
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    
    # Performance tuning for M3 Max 
    con.execute("SET threads=16;")  # M3 Max with 16 perf. cores
    con.execute("SET memory_limit='16GB';")  # Dont even need all that 128Gb RAM!
    con.execute("SET max_memory='16GB';")
    con.execute("SET temp_directory='./tmp';") 
    con.execute("SET enable_progress_bar=true;")
    
    # Parallel processing optimizations
    con.execute("SET preserve_insertion_order=false;")  # Allow reordering for performance
    
    print("Running optimized geometry construction query...")
    df = con.sql(query).to_df()

#print(f"Query completed! Retrieved {len(df)} construction roads.")
df 

# 13.3s for all of Germany, results in 10048 rows

Connecting to DuckDB with optimized settings...
Running optimized geometry construction query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id,tags,geom_wkb
0,10661306,"{'check_date': '2025-05-31', 'construction': '...","[1, 2, 0, 0, 0, 3, 0, 0, 0, 230, 27, 27, 203, ..."
1,24216641,"{'access': 'no', 'construction': 'service', 'e...","[1, 2, 0, 0, 0, 20, 0, 0, 0, 164, 207, 162, 21..."
2,32440576,"{'bridge': 'yes', 'construction': 'secondary',...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 71, 34, 52, 130, 1..."
3,35341293,"{'access': 'no', 'check_date:lit': '2021-04-28...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 3, 31, 205, 162, 1..."
4,42815874,"{'check_date': '2024-07-23', 'construction': '...","[1, 2, 0, 0, 0, 6, 0, 0, 0, 23, 233, 143, 122,..."
...,...,...,...
10044,1355384437,"{'bridge': 'yes', 'construction': 'footway', '...","[1, 2, 0, 0, 0, 4, 0, 0, 0, 124, 28, 188, 101,..."
10045,1355384439,"{'construction': 'footway', 'footway': 'sidewa...","[1, 2, 0, 0, 0, 3, 0, 0, 0, 39, 52, 73, 44, 41..."
10046,1371921132,"{'construction': 'service', 'highway': 'constr...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 137, 50, 10, 56, 5..."
10047,1376547400,"{'bridge': 'yes', 'construction': 'motorway', ...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 42, 37, 95, 100, 9..."


In [9]:
import duckdb
import geopandas as gpd
import pandas as pd
import os

# --- 1. Setup ---
pbf_file = "planet-250602.osm.pbf"

if not os.path.exists(pbf_file):
    raise FileNotFoundError(f"The file '{pbf_file}' was not found.")

# --- 2. Optimized High-Performance DuckDB Query ---
query = f"""
PRAGMA enable_optimizer;
WITH
    -- Step 1: Filter ways based on tags first. This is the most selective filter.
    -- The optimizer will push this down to the OSM reader.
    filtered_ways AS (
        SELECT id, tags, refs
        FROM st_readOSM('{pbf_file}')
        WHERE
            kind = 'way'
            AND tags['highway'] IS NOT NULL
            AND tags['construction'] IS NOT NULL
            -- It's good practice to also exclude proposed/planned features if not wanted
            -- AND tags['construction'] NOT IN ('proposed', 'planned')
            AND array_length(refs) >= 2
    ),

    -- Step 2: Get only the nodes that are referenced by our filtered ways.
    -- This is more direct than creating a separate CTE for IDs.
    nodes AS (
        SELECT id, ST_Point(lon, lat) AS geom
        FROM st_readOSM('{pbf_file}')
        WHERE
            kind = 'node'
            AND id IN (SELECT UNNEST(refs) FROM filtered_ways)
    ),
    
    -- Step 3: Unnest way-node relationships for ordering
    way_nodes AS (
        SELECT
            w.id AS way_id,
            w.tags,
            UNNEST(w.refs) AS node_id,
            UNNEST(generate_series(1, array_length(w.refs))) AS node_order
        FROM filtered_ways w
    )

-- Step 4: Final geometry construction
SELECT
    wn.way_id AS id,
    wn.tags,
    ST_AsWKB(ST_MakeLine(LIST(n.geom ORDER BY wn.node_order))) AS geom_wkb
FROM way_nodes wn
INNER JOIN nodes n ON wn.node_id = n.id
GROUP BY wn.way_id, wn.tags;
"""

# --- 3. Execute with Optimized Configuration ---
print("Connecting to DuckDB with optimized settings...")

with duckdb.connect(database=':memory:') as con:
    # Optimize DuckDB for M3 Max performance
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    
    # Performance tuning 
    con.execute("SET threads=16;") 
    con.execute("SET memory_limit='16GB';")  # more RAM yields no benefit
    con.execute("SET max_memory='16GB';")
    con.execute("SET temp_directory='./tmp';") 
    
    # Enable query optimization
    #con.execute("SET disabled_optimizers=true;")
    #con.execute("SET enable_profiling=true;")
    con.execute("SET enable_progress_bar=true;")
    
    # Parallel processing optimizations
    con.execute("SET preserve_insertion_order=false;")  # Allow reordering for performance
    
    print("Running optimized geometry construction query...")
    df = con.sql(query).to_df()

#print(f"Query completed! Retrieved {len(df)} construction roads.")
df 

# 13.7s for Germany!
# 4m 12.5s for the entire planet with 349543 rows!

Connecting to DuckDB with optimized settings...
Running optimized geometry construction query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id,tags,geom_wkb
0,531084270,"{'construction': 'unclassified', 'highway': 'c...","[1, 2, 0, 0, 0, 4, 0, 0, 0, 80, 158, 195, 198,..."
1,531084309,"{'construction': 'tertiary', 'highway': 'const...","[1, 2, 0, 0, 0, 18, 0, 0, 0, 3, 133, 213, 179,..."
2,531084343,"{'construction': 'trunk_link', 'highway': 'con...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 55, 9, 148, 3, 44,..."
3,531084349,"{'construction': 'unclassified', 'highway': 'c...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 211, 253, 156, 130..."
4,531084356,"{'construction': 'tertiary', 'highway': 'const...","[1, 2, 0, 0, 0, 3, 0, 0, 0, 157, 228, 253, 110..."
...,...,...,...
349539,1390219101,"{'check_date': '2025-05-27', 'construction': '...","[1, 2, 0, 0, 0, 4, 0, 0, 0, 247, 57, 227, 160,..."
349540,1391444781,"{'check_date': '2024-08-02', 'construction': '...","[1, 2, 0, 0, 0, 3, 0, 0, 0, 1, 169, 77, 156, 9..."
349541,1391450513,"{'access': 'no', 'check_date': '2025-06-01', '...","[1, 2, 0, 0, 0, 2, 0, 0, 0, 126, 231, 23, 37, ..."
349542,1391459347,"{'check_date': '2025-05-31', 'construction': '...","[1, 2, 0, 0, 0, 4, 0, 0, 0, 29, 177, 22, 159, ..."


In [ ]:
# if you need additional filters, brute forcing a string-based filter without index is usually fast enough
# df[df["tags"].apply(lambda x: str(x)).str.contains("construction")]


# 2. Convert to Geopandas gdf

In [10]:
gdf = gpd.GeoDataFrame(
# df # you can add the other columns here if needed, but note that it will lead to unexplainable errors when using the data in deckgl
# it's awful to debug, so best just keep the columns you actually need which is the geometry
geometry=gpd.GeoSeries.from_wkb(df['geom_wkb'].apply(bytes)),
        crs="EPSG:4326"
    )

In [11]:
# gdf["tags"] = gdf["tags"].apply(lambda x: str(x))
# converting to string does not fix the deckgl error, needs investigation...

In [12]:
gdf # 10k rows

,geometry
0,"LINESTRING (14.656 36.952, 14.656 36.951, 14.6..."
1,"LINESTRING (14.667 37.012, 14.667 37.012, 14.6..."
2,"LINESTRING (14.641 37.085, 14.641 37.085)"
3,"LINESTRING (14.65 37.115, 14.65 37.115)"
4,"LINESTRING (14.673 37.148, 14.673 37.148, 14.6..."
...,...
349539,"LINESTRING (71.581 30.305, 71.581 30.305, 71.5..."
349540,"LINESTRING (-70.599 -33.62, -70.6 -33.62, -70...."
349541,"LINESTRING (-80.342 41, -80.342 41)"
349542,"LINESTRING (-60.82 -32.938, -60.82 -32.938, -6..."


# 3. Visualize with Deck.gl

In [ ]:
import pydeck as pdk

gdf['path_coords'] = gdf['geometry'].apply(lambda geom: [list(coord) for coord in geom.coords])

INITIAL_VIEW_STATE = pdk.ViewState(latitude=47.65, longitude=7, zoom=4.5, max_zoom=16)#pitch=50, bearing=0)

line_layer = pdk.Layer(
    "PathLayer",  # This is the correct layer for polylines (multi-point lines)
    gdf,          # Your GeoDataFrame
    get_path="path_coords",
        get_color=[255, 100, 100, 200], # Red from your style.json (RGBA, 200 for 80% opacity)
        get_width_min_pixels=3,         # Minimum pixel width for the lines (from line-width)
        get_dash_array=[3, 3],          # From your style.json line-dasharray
        width_scale=1,                  # Optional: scale for width
        width_min_pixels=1,             # Optional: ensures lines are visible at very low zoom
        width_max_pixels=100,           # Optional: prevents lines from becoming excessively thick
        highlight_color=[255, 255, 0],  # Yellow highlight on hover
        picking_radius=50,
        auto_highlight=True,
        pickable=True,
)

layers = [line_layer]

r = pdk.Deck(layers=layers, initial_view_state=INITIAL_VIEW_STATE)


TypeError: deck_to_html() got an unexpected keyword argument 'display'

In [ ]:
r.to_html("line_layer.html",  notebook_display=False) # too large, 500Mb!